<div><div style="text-align:left;display:inline-block;float:left"><code>Eduardo Cardenas</code></div> <div style="text-align:right;display:inline-block;float:right"><code>Computación Científica - UNMSM</code></div></div>
<hr width=100% align=center>
<h1 style="text-align:center">Método Numerico - Ecuación de Poisson</h1>

$$
\begin{cases}
 \dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2}= f(x,y)  & \text{en } \Omega = (a,b)\times(c,d)\\ 
 {\scriptsize\mbox{ tal que}} & \\ 
 u(x,y) = g(x,y) & \text{para } (x,y)\in \partial \Omega    \\
\end{cases} 
$$

Denotamos la siguiente partición:
$$h = \frac{b-a}{n}, \qquad k = \frac{c-d}{m} \quad \mbox{tal que} \quad x_{i+1} = x_i + h, \qquad y_{j+1} = y_j + k $$

Utilizamos la siguiente notación :
$$ w_{i,j} = u(x_i,t_j) $$

### Formulas para aproximar las derivadas parciales
Utilizamos las siguientes formulas : 

$$ \frac{\partial^2 u}{\partial x^2} \simeq \frac{w_{i+1,j} -2w_{i,j} + w_{i-1,j}}{h^2}$$

$$ \frac{\partial^2 u}{\partial y^2} \simeq \frac{w_{i,j+1} -2w_{i,j} + w_{i,j-1}}{k^2}$$

### Formula para la Ecuación de Poisson

Conocidas las aproximaciones de las derivadas parciales , reemplazemos sus aproximaciones en la ecuación de Po 

$$ \frac{w_{i+1,j} -2w_{i,j} + w_{i-1,j}}{h^2} + \frac{w_{i,j+1}- 2w_{i,j} + w_{i,j-1}}{k^2} = f(x_i,y_i)$$ 
Sea $\lambda = \dfrac{h^2}{k^2}  $ :
$$w_{i-1,j} - 2w_{i,j} + w_{i+1,j} + \lambda w_{i,j-1} -2\lambda w_{i,j} +\lambda w_{i,j+1} = h^2f(x_i,y_i)$$

$$w_{i-1,j} + w_{i+1,j} -2(1+\lambda)w_{i,j} + \lambda w_{i,j-1} + \lambda w_{i,j+1} = h^2f(x_i,y_i)$$

Tomamos la siguiente malla de puntos:

![title](vista.png)

$$
\scriptsize
\begin{array} 
~-2(1+\lambda)w_{1,4} & + & w_{2,4} & + & 0 & + & 0 & + & 0 & + & \lambda w_{1,3} & & & & & & & & &  = h^2 f_{1,4}-w_{0,4}-\lambda w_{1,5} \\
w_{1,4} & - & 2(1+\lambda)w_{1,4} & + & w_{3,4} & + & 0 & + & 0 & + & 0 & + &\lambda w_{2,3} & & & & & & &  = h^2 f_{2,4}-\lambda w_{2,5} \\
0 & + & w_{2,4} & - & 2(1+\lambda)w_{3,4} & + & w_{4,4} & + & 0 & + & 0 & + & 0 & + & \lambda w_{3,3} & & & & &  = h^2 f_{3,4} - \lambda w_{3,5} \\
0 & + & 0 & + & w_{3,4} & - & 2(1 + \lambda) w_{4,4} & + & w_{5,4} & + & 0 & + & 0 & + & 0 & + & \lambda w_{4,3} & & &  = h^2 f_{4,4} - \lambda w_{4,5} \\
\vdots & & \vdots & & \vdots & & \vdots & & \vdots & & \vdots & & \vdots & & \vdots  & & \ddots & & & = \vdots
\end{array}
$$

### Código Python

In [5]:
import numpy as np

def tridiag(a,b,c,N):
    A = np.zeros((N,N))
    
    np.fill_diagonal(A[:-1,1:],a)
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],c)
    
    return A

def metodoEliptica(a, b, c, d, h, k, f, g_a, g_b, g_c, g_d):
    
    """
    a,b : x in (a,b)
    c,d : y in (c,d)
    h,k : pasos de la partición para x , y
    f : f(x,y)
    g_a : función limite izquiero
    g_b : función limite derecho
    g_c : función limite inferio
    g_d : función limite superior
    """
    
    r = (h**2)/k**2

    n = round((b-a)/h) + 1
    m = round((d-c)/k) + 1

    x = np.linspace(a , b, n);
    y = np.linspace(c , d, m);

    p = (n-2)*(m-2)
     
    C = tridiag(r, -2*(1+r), r, n-2)

    A = np.kron(np.eye(m-2), C)
    
    np.fill_diagonal(A[:(p-(n-2)),(n-2):],r)

    np.fill_diagonal(A[(n-2):,:(p-(n-2))],r)

    xx,yy = np.meshgrid(x[1:-1],y[1:-1])
    
    B = np.zeros((m-2,n-2))
    
    for i in range(m-2):
        for j in range(n-2):
            B[i,j] = (h**2) * f(x[j+1], y[-(i+2)]) 

    for i in range(m-2):  
        B[i,0] -= g_a( y[-(i+2)] )
        B[i,-1] -= g_b( y[-(i+2)] )         

    for j in range(n-2):
        B[0,j] -= r*g_d(x[j+1])
        B[-1,j] -= r*g_c(x[j+1]) 
    
    B = np.reshape(B, np.size(B))

    W = np.linalg.solve(A, B)

    #malla de solucion

    sol = np.zeros((n,m))

    sol[0,:]  = [g_d(i) for i in x]
    sol[-1,:] = [g_c(i) for i in x]
    sol[:,0]  = [g_a(i) for i in y]
    sol[:,-1] = [g_b(i) for i in y]

    W = np.reshape(W, (m-2,n-2) )    

    sol[1:-1,1:-1] = W

    return sol

### Ejercicio (Análisis Matemático - Burden - pag 719)

$$
\begin{cases}
 \dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2}= 0  & \text{en } \Omega = (0,0.5)\times(0,0.5)\\ 
 {\scriptsize\mbox{ tal que}} & \\ 
 u(0,y) = 0 & u(x,0) = 0 \\
 u(x,0.5) = 200x & u(0.5,y) =200y 
\end{cases} 
$$

Determinar la solución para $h=k=0.125$

### Solución

Utilizamos el metodo desarrollado para aproximar la solución

In [13]:
sol_aprox = metodoEliptica(0, 0.5, 0, 0.5, 0.125, 0.125, lambda x, y: 0,
                                                         lambda x: 0, 
                                                         lambda x: 200*x,
                                                         lambda x: 0,
                                                         lambda x: 200*x)

In [14]:
sol_aprox

array([[  0.  ,  25.  ,  50.  ,  75.  ,   0.  ],
       [  0.  ,  18.75,  37.5 ,  56.25,  25.  ],
       [  0.  ,  12.5 ,  25.  ,  37.5 ,  50.  ],
       [  0.  ,   6.25,  12.5 ,  18.75,  75.  ],
       [  0.  ,   0.  ,   0.  ,   0.  , 100.  ]])